In [ ]:
# ===== Rough Flask App #1: Minimal JSON API =====
# Use in Colab. Starts an ngrok URL like your original.
# Endpoints:
#   GET  /api/health
#   GET  /api/diseases
#   POST /api/predict  (JSON: dish, age, disease)

!pip install -q flask flask-ngrok

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from mistral_llm_cpu import generate_recipe_output_with_trust, all_diseases

app_api = Flask(__name__)
run_with_ngrok(app_api)

@app_api.get("/api/health")
def health_080():
    return jsonify({"status": "ok", "app": "NutriFusion JSON API"})

@app_api.get("/api/diseases")
def diseases_080():
    # returns the list you already expose in your main app
    return jsonify({"diseases": all_diseases})

@app_api.post("/api/predict")
def predict_080():
    """
    Expected JSON body:
    {
      "dish": "Paneer Tikka",
      "age": 45,
      "disease": "diabetes"
    }
    """
    try:
        data = request.get_json(force=True)
        dish = data.get("dish", "").strip()
        age = int(data.get("age", 0))
        disease = data.get("disease", "").strip()

        output, _, trust_score, explanation = generate_recipe_output_with_trust(dish, age, disease)

        return jsonify({
            "dish": dish,
            "age": age,
            "disease": disease,
            "output": output,
            "trust_score": trust_score,
            "explanation": explanation
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 400

app_api.run()
